In [1]:
import pandas
import scipy.io as sio

# include exp and log functions
import math
import numpy as np

# Use this to return multivariable in one function
import collections

import time, json

In [2]:
# load matlab data.
mat = sio.loadmat('data/spam.mat')

In [3]:
_train = mat['train_spam']
# train[:,-1]

In [4]:
_test = mat['test_spam']
len(_test)

1000

In [5]:
def update_weight(point, weight, alpha, f_t):
    '''
    This function takes in a observation point, its original weight and the current calculated alpha and
    its weak learning function A. This function returns the new weight for this point.
    This function should be called by using map/comprehension.
    
    point: a single column with length = feature_count + 1, the last one is the label
    weight: should be a single value smaller than one, and normalized.
    alpha: a value calculated with zt during every loop of adaboost.
    f_t: a tuple with four elements (feature_index, feature_theta, left_node_class, right_node_class)
    '''
    
    f_idx = f_t[0]
    f_theta = f_t[1]
    left_node_class = f_t[2]
    right_node_class = f_t[3]
        
    label = point[-1]
    if label == 0:
        label = -1
        
    if point[f_idx] <= f_theta:
        # classified to the left hand side node
        classified_as = left_node_class
    else:
        classified_as = right_node_class
        
    new_weight = weight * math.exp((-1) * alpha * label * classified_as)
    
    return new_weight
    

In [6]:
# try different alpha approach.
# compare results with sklearn trained dt
# last one can't be skipped... or not?

def find_dt(train_data, weight):
    '''
    find_dt means find decision tree. For this function, we find a one level decision tree
    
    train_data: is the dataframe given into the function
    weight: is an array the same length of the number of data points, and should be updated everytime before passing
    into this function.
    
    returns feature index, theta value(threshold), left_node_class, right_node_class 
    '''
    
    feature_count = train_data.shape[1] - 1
    
    # loop through all features
    costs = []
    for idx in reversed(range(feature_count)):
        features = train_data[:, idx]
        labels = train_data[:, -1]
        f_temp = np.vstack((features, labels, weight)).T
        f_temp = f_temp[f_temp[:, 0].argsort(),:]
        
        # print(f_temp[:,0])
        feature_costs = []
        for idx, f_val in enumerate(f_temp[:,0]):
            pass
            
            left_node_class, left_node_cost = 0, 1000
            right_node_class, right_node_cost = 0, 1000
        
            if idx == len(f_temp) - 1:
                continue
            
            if f_val == f_temp[idx+1 , 0]:
                continue
                
            # if it's not the same as the next value, we split by its value.
            # print(f_temp[0:idx+1,:])
            # if left node is classified as zero
            left_zero_cost = np.dot(f_temp[0:idx+1,1], f_temp[0:idx+1,2])
            left_one_cost  = np.dot((f_temp[0:idx+1,1] -1)*(-1), f_temp[0:idx+1, 2])
            right_zero_cost = np.dot(f_temp[idx+1:, 1], f_temp[idx+1:, 2])
            right_one_cost  = np.dot( (f_temp[idx+1:, 1] -1)*(-1), f_temp[idx+1:, 2])
    
            if left_zero_cost <= left_one_cost:
                left_node_class = -1
                left_node_cost = left_zero_cost
            else:
                left_node_class = 1
                left_node_cost = left_one_cost
            
            if right_zero_cost <= right_one_cost:
                right_node_class = -1
                right_node_cost = right_zero_cost
            else:
                right_node_class = 1
                right_node_cost = right_one_cost
            
            cost = left_node_cost + right_node_cost
            f_t = (cost, f_val, left_node_class, right_node_class)
            feature_costs.append(f_t)
        
        # if all the points have the same value for this feature, we'll need this part
        # that adds fake data into the list, so that the index won't be messed up when we want to find the
        # feature that gives the smallest weighted cost.
        if not feature_costs:
            feature_costs.append((100, 0, 0,0))
            
        min_f = min(feature_costs, key=lambda x: x[0])
        costs.append(min_f)
    
    # after looping through all the features, we look for the feature that gives us the smallest cost?
    min_cost = min(costs, key=lambda x:x[0])
    f_idx = costs[::-1].index(min_cost)

    d_t = (f_idx,) + min_cost[1:]
#     print('finding the best decision tree...')
#     print(d_t)
#     print('')
    return d_t
            
            
                

In [12]:
def hw3_train_adaboost(train_data, num_rounds):
    
    # init variables to loop through
    n = len(train_data)
    features_count = len(train_data[0]) - 1 
    
    # init the weights for each data point
    weights = [1/n] * n
    
    
    # init the input vars to match my function's var names
    T = num_rounds
    train = train_data
    
    
    # two list that saves all the alpha values and the weak function for each iteration
    alphas = []
    # this is a list that saves tuple with four element in each element
    # (idx, theta, left_node_class, right_node_class)
    # [(1,2,3,4), (1,2,3,4)...]
    f_ts = []

    
    # make an idx list so we can reference back to the weights list
    idx_list = [x for x in range(n)]
    

    # do T times
    # loop through observations that finds the best 1 layer decision tree 
    for t in range(0, T):

        # find the one level decision tree here
        dt = find_dt(train, weights)
        
        feature_theta = dt[1]
        feature_idx = dt[0]
        left_node_class = dt[2]
        right_node_class = dt[3]
        
        #######################################
        # The Simple classifier is found here #
        #######################################
        
#         print('========= DONE =========')
#         print('f_idx: ' + str(feature_idx))
#         print('f_theta: ' +  str(feature_theta) )
#         print('left node class= ' + str(left_node_class))
#         print('right node class= ' + str(right_node_class))
#         print('')
        
        # update zt
        zt = 0
        for p_idx, point in enumerate(train[:, feature_idx]):
            label = train[p_idx, -1]
            # to calculate zt, we need to take 0 as -1, but keep the one.
            if label == 0:
                label = -1
                
            if point <= feature_theta:
                # this will go to left node.
                zt = zt + weights[p_idx] * (left_node_class * label)
                # print('added ' + str(weights[p_idx] * (left_node_class * label)) )
            else:
                zt = zt + weights[p_idx] * (right_node_class * label)
                # print('added ' + str(weights[p_idx] * (right_node_class * label)) )
        
        
        # print('zt: ' + str(zt))
                
        # calculate alpha value for this loop
        alpha = (1/2)* math.log((1 + zt)/(1 - zt))
        # print('alpha: ' + str(alpha))
        
        alphas.append(alpha)

        # append this function
        f_t = (feature_idx, feature_theta, left_node_class, right_node_class)
        f_ts.append((feature_idx, feature_theta, left_node_class, right_node_class))

        # update weights
        weights = [update_weight(train[w_idx, :], w, alpha, f_t) for w_idx, w in enumerate(weights)]
        
        # normalize weights
        weight_sum = sum(weights)
        weights = [float(x)/weight_sum for x in weights]
        # print('sum of weights: ' + str(sum(weights)))
        
    parameters = collections.namedtuple('Parameters', ['alphas', 'functions'])
    params = parameters(alphas, f_ts)
    return params



In [458]:
trained_params = hw3_train_adaboost(_train, 1)
print(hw3_test_adaboost(trained_params, _test))

trained_params = hw3_train_adaboost(_train, 2)
print(hw3_test_adaboost(trained_params, _test))

trained_params = hw3_train_adaboost(_train, 4)
print(hw3_test_adaboost(trained_params, _test))

trained_params = hw3_train_adaboost(_train, 8)
print(hw3_test_adaboost(trained_params, _test))

trained_params = hw3_train_adaboost(_train, 16)
print(hw3_test_adaboost(trained_params, _test))

trained_params = hw3_train_adaboost(_train, 32)
print(hw3_test_adaboost(trained_params, _test))

trained_params = hw3_train_adaboost(_train, 64)
print(hw3_test_adaboost(trained_params, _test))

trained_params = hw3_train_adaboost(_train, 100)
print(hw3_test_adaboost(trained_params, _test))

192
192
135
67
63
46
43
39


In [25]:
def loss_of_data_set(train, test, num_round):
    # num_rounds = [x for x in range(1, 101)]
    loss = []
    
    trained_params = hw3_train_adaboost(train, num_round)
    alphas = trained_params.alphas
    f_ts = trained_params.functions
    
    
    
    for t in range(num_round):
        
        parameters = collections.namedtuple('Parameters', ['alphas', 'functions'])
        params = parameters(alphas[0:t+1], f_ts[0:t+1])
        l = hw3_test_adaboost(params, test)
        loss.append(l)
    
    return loss
    print(loss)

In [26]:
# training error
# train_error = loss_of_data_set(_train, _train, 100)

test_error = loss_of_data_set(_train, _test, 100)

# print(train_error)
print(test_error)

[192, 192, 61, 135, 58, 64, 57, 67, 50, 65, 51, 65, 58, 56, 61, 63, 51, 63, 50, 61, 50, 57, 49, 49, 48, 45, 46, 48, 44, 47, 43, 46, 43, 44, 51, 43, 51, 43, 50, 47, 45, 48, 44, 47, 43, 48, 43, 47, 43, 44, 42, 46, 42, 45, 44, 44, 46, 44, 43, 44, 43, 44, 43, 43, 43, 43, 42, 42, 42, 41, 44, 41, 43, 42, 39, 43, 39, 43, 39, 43, 40, 43, 39, 42, 39, 41, 40, 40, 40, 40, 40, 40, 40, 39, 39, 40, 39, 39, 39, 39]


In [27]:
# training error
train_error = loss_of_data_set(_train, _train, 100)
print(train_error)

[596, 596, 180, 434, 185, 211, 176, 213, 148, 202, 155, 187, 154, 172, 172, 190, 150, 181, 142, 176, 142, 167, 144, 145, 151, 138, 139, 151, 131, 149, 130, 142, 127, 130, 145, 129, 142, 125, 135, 133, 127, 135, 125, 130, 125, 132, 122, 133, 117, 127, 122, 131, 121, 129, 124, 121, 120, 124, 116, 123, 113, 120, 114, 112, 116, 119, 114, 117, 113, 108, 113, 104, 113, 109, 108, 111, 107, 109, 105, 107, 106, 107, 106, 108, 107, 108, 106, 110, 105, 107, 105, 107, 105, 106, 104, 108, 107, 110, 107, 108]


In [426]:
_train[0:5, 54:]

print(_train[0:5, 54:])
find_dt(_train[0:5, 54:], np.ones(5)/5)

[[   1.394    6.     159.       1.   ]
 [   1.567    6.     428.       0.   ]
 [   3.222    9.      29.       0.   ]
 [   2.815   26.     107.       1.   ]
 [   4.971   76.     517.       0.   ]]
costs: 
[(0.20000000000000001, 159.0, 1, -1), (0.40000000000000002, 6.0, -1, -1), (0.20000000000000001, 1.3939999999999999, 1, -1)]


(2, 159.0, 1, -1)

<h1>Plot for Part 2 here</h1>

In [29]:
import matplotlib.pyplot as plt
from scipy.interpolate import spline

In [18]:
len(_test)
len(_train)

3601

In [ ]:
# test data
num_rounds = np.array([x for x in range(1,101)]) # x-axis
loss = np.array([192, 192, 135, 67, 63, 46, 43, 39]) # y-axis

#training data
train_loss = np.array([596, 596, 434, 213, 190, 142, 112, 108])

# xnew = np.linspace(num_rounds.min(), num_rounds.max(), 300)
# loss_smooth = spline(num_rounds, loss, xnew)

# plt.figure()
fig, ax = plt.subplots()
ax.plot(num_rounds, loss/1000, '-' , label='test')
ax.plot(num_rounds, train_loss/3601, '--', label='train')
ax.set_xlabel('num rounds')
ax.set_ylabel('error rate')

legend = ax.legend(loc='upper center', shadow=True)
plt.show()

In [9]:
def classify(function, point):
    '''
    This class classifies the result of a point by using the given function, a weak function.
    function: a 4 elements tuple, 1.feature_index, 2.theta value, 3.left node class, 4. right node class
    returns one of the values of left or right.
    '''
    
    f_idx = function[0]
    f_theta = function[1]
    left_node_class = function[2]
    right_node_class = function[3]
    
    if point[f_idx] <= f_theta:
        return left_node_class
    else:
        return right_node_class
    

In [10]:
def hw3_test_adaboost(params, test_data):
    alphas = params.alphas
    functions = params.functions
    loss = 0
    
    for td in test_data:
        # print(td)
        
        sum = 0
        for idx, f in enumerate(functions):
            # feature_idx, feature_theta, left_node_class, right_node_class
            sum = sum + alphas[idx] * classify(f, td)
        
        if sum * td[-1] < 0:
            loss = loss + 1
            
    return loss
    

In [26]:
a = np.array([1,2,3,4])
b = np.array([0,2,0,1])
np.dot(a,b)


array([1, 4, 3, 5])

In [29]:
np.arange(10).reshape(2,5).T

array([[0, 5],
       [1, 6],
       [2, 7],
       [3, 8],
       [4, 9]])

In [37]:
c = np.vstack((a,b))

In [43]:
c[0][1]

2

In [40]:
c[1]

array([0, 2, 0, 1])

In [54]:
z = np.array([[1,4,3], [3,1,2], [1,2,3]])
z[0]

array([1, 4, 3])

In [56]:
np.sort(z, axis=1)

array([[1, 3, 4],
       [1, 2, 3],
       [1, 2, 3]])

In [57]:
np.vstack(([1,2,3],[2,3,4]))

array([[1, 2, 3],
       [2, 3, 4]])

In [94]:
a = np.array([1,2,3,4,5])
b = np.array([0,-1,0,-1,0])

In [96]:
c = np.vstack((a,b))

In [102]:
print(c)
c[:,0]

[[ 1  2  3  4  5]
 [ 0 -1  0 -1  0]]


array([1, 0])

In [103]:
len(c)

2

In [262]:
for x in _train[:, 5]:
    if x != 0:
        print(x)

0.14
0.78
0.16
0.32
0.57
0.56
0.04
0.07
0.65
0.09
0.65
0.6
0.16
0.51
0.11
0.17
0.31
0.58
0.12
0.19
0.21
0.09
0.32
0.46
0.53
0.63
0.64
0.65
0.36
0.15
0.19
0.62
0.03
0.47
0.21
0.17
0.4
0.17
0.18
0.04
0.08
0.19
0.24
0.17
0.05
0.32
0.35
0.1
0.12
0.53
0.13
0.19
0.87
0.36
0.13
0.64
0.2
0.19
0.1
0.2
0.29
0.6
0.1
0.05
0.34
0.2
0.19
0.8
0.03
0.05
0.54
0.67
1.02
1.26
0.38
0.11
0.6
0.44
0.35
0.23
0.16
0.46
0.11
0.1
0.36
0.32
0.27
0.44
0.58
0.09
2.63
0.09
0.05
0.12
0.6
0.45
0.35
0.45
0.8
0.21
0.4
0.73
0.61
0.08
0.98
0.1
0.21
0.38
0.16
0.44
0.3
0.61
0.16
0.25
0.16
0.61
0.99
0.44
0.98
0.21
0.1
0.22
0.25
0.58
0.59
0.47
0.27
0.21
0.05
0.26
0.16
1.09
0.33
0.22
0.65
0.9
0.57
0.19
0.57
0.57
0.42
0.13
0.4
0.17
0.26
0.05
0.11
0.12
0.67
0.09
0.18
0.03
1.01
1.02
1.12
0.23
0.36
0.77
0.1
0.1
0.5
0.33
0.24
0.17
1.29
0.29
0.68
0.25
0.36
0.1
0.24
0.03
0.17
0.32
0.3
0.8
0.12
0.27
0.11
0.22
0.8
0.26
0.23
0.5
0.99
1.02
0.34
0.55
1.02
0.22
0.06
0.13
1.06
0.84
1.21
0.65
0.18
1.27
0.49
0.49
0.46
0.58
0.62
0.37
3.44
0.5

In [319]:
import sklearn.tree
len(_train[0])
_train[0, 0:58]

array([  0.00000000e+00,   5.70000000e-01,   5.70000000e-01,
         0.00000000e+00,   1.40000000e-01,   1.40000000e-01,
         0.00000000e+00,   0.00000000e+00,   1.40000000e-01,
         0.00000000e+00,   0.00000000e+00,   4.30000000e-01,
         1.40000000e-01,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   1.40000000e-01,   0.00000000e+00,
         3.31000000e+00,   0.00000000e+00,   1.44000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   5.70000000e-01,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,

In [316]:
h = sklearn.tree.DecisionTreeClassifier(max_depth=1)

In [325]:
ws = np.ones(10)/10
h.fit( _train[0:10,0:57], _train[0:10, 57], sample_weight=ws )
pred = h.predict(_train[0:10, 0:57])
print(train[0:10, 57])
pred


[ 1.  0.  0.  1.  0.  1.  0.  0.  0.  0.]


array([ 1.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  1.,  0.])

In [337]:
print(h.tree_.children_left)
print(h.tree_.children_right)
print(h.tree_.feature)
print(h.tree_.threshold)

[ 1 -1 -1]
[ 2 -1 -1]
[51 -2 -2]
[ 0.0385 -2.     -2.    ]
